In [130]:
import pandas as pd
import numpy as np

# df = pd.read_parquet("products_descr.parquet")
df = pd.read_parquet("products_descr_sample.parquet")

df['new_descr'] = df['BRAND_NAME'] + " " + df['ITEM_DESCRIPTION']
df['descr'] = " " + df['new_descr'].str.lower().str.strip() + " "

In [113]:
file_tax = "short_taxonomy.txt"
taxonomy = open(file_tax,"rt").readlines()


## removing non-text symbols
taxonomy = [x.replace("\n","") for x in taxonomy]

## removing comments
## removing text after '#' symbol
taxonomy = [x.split("#")[0] for x in taxonomy]

## removing empty lines
taxonomy = [x for x in taxonomy if len(x)>0 ]

## lowering lines
taxonomy = [x.lower() for x in taxonomy if len(x)>0 and x[0]!="#" ]


# ## last portion of request contains query that we will search
# ## (query, # levels in category, category description)
taxonomy = [[y.strip() for y in x.split("->")] for x in taxonomy ]

## removing taxonomy rows which doesn't have filters (at least one -> )
taxonomy = [x for x in taxonomy if len(x)>1]

## only gender related queries
taxonomy_gen = [x for x in taxonomy if x[0] == "gender"]
taxonomy_gen = [(x[-1], 1, x[1]) for x in taxonomy_gen]


## only non-gender queries
taxonomy = [x for x in taxonomy if x[0] != "gender"]
taxonomy = [(x[-1], len(x)-1, " -> ".join(x[:-1])) for x in taxonomy]




In [121]:
%%time

import taxonomy_utils 
import taxonomy_search
from importlib import reload


reload(taxonomy_search)
reload(taxonomy_utils)


get_non_repeated_words = taxonomy_utils.get_non_repeated_words
get_words_cache_dict = taxonomy_utils.get_words_cache_dict
query_words = taxonomy_utils.query_words




## list of words that doesn't repeat in any other category
words_non_repeated = get_non_repeated_words(taxonomy)
dict_cache = get_words_cache_dict(taxonomy, is_cache)
    
    

taxonomy_search.simple_search(df, 
                              taxonomy,taxonomy_gen,
                              words_non_repeated = words_non_repeated,
                              dict_cache = dict_cache)

Count words to cache 155
Total rules 190
Non matched products 8.5 %
Non matched gender/age 82.9 %
Delta time 0s
CPU times: user 18.2 s, sys: 331 ms, total: 18.5 s
Wall time: 18.7 s


,COUNTRY,BRAND_NAME,ITEM_GROUP,mc_code,ITEM_CODE,BIC_ZBUSUNIT,BIC_ZBUYGROUP,MERCHANDISE_CATEGORY_TEXT,ITEM_DESCRIPTION,level_0_txt,...,trans_val,category_name,new_descr,descr,taxonomy,tag_weight,filt,query,profile,tax_weight
0,United Arab Emirates,ACE,02cc3ca9-ea79-4e1b-b387-ab25f58afde4,,02cc3ca9-ea79-4e1b-b387-ab25f58afde4,None,None,None,Orchid Wooden Kitchen Tissue Holder (27.7 x 13...,None,...,0.0,non_classified,ACE Orchid Wooden Kitchen Tissue Holder (27.7 ...,ace orchid wooden kitchen tissue holder (27.7...,home -> storage & organization,4,None,tissue holder,None,4
1,United Arab Emirates,ACE,47e77dcd-1abc-40d4-a2e5-816185135db6,,47e77dcd-1abc-40d4-a2e5-816185135db6,None,None,None,Mr. Clean Magic Eraser Sponge For Bathroom Pac...,None,...,0.0,non_classified,ACE Mr. Clean Magic Eraser Sponge For Bathroom...,ace mr. clean magic eraser sponge for bathroo...,home -> utils,3,None,sponge,None,3
2,United Arab Emirates,ACE,9e108baf-a919-4123-8444-cb55b7df56b8,,9e108baf-a919-4123-8444-cb55b7df56b8,None,None,None,Oasis Casual Rattan Dining Chair W/Cushion (58...,None,...,4.0,non_classified,ACE Oasis Casual Rattan Dining Chair W/Cushion...,ace oasis casual rattan dining chair w/cushio...,furniture -> chairs,1,None,chair,None,1
3,United Arab Emirates,ACE,a4e95c3d-1832-4539-b847-dfa221b5625d,,a4e95c3d-1832-4539-b847-dfa221b5625d,None,None,None,"Geepas Rechargeable Emergency LED Lantern Set,...",None,...,0.0,non_classified,ACE Geepas Rechargeable Emergency LED Lantern ...,ace geepas rechargeable emergency led lantern...,home -> ligths,3,None,led,None,3
4,United Arab Emirates,ACE,b70ad26e-9808-49d8-b489-3ce4905f275e,,b70ad26e-9808-49d8-b489-3ce4905f275e,None,None,None,Bergner Ion Non-Stick Stainless Steel Knife Se...,None,...,0.0,non_classified,ACE Bergner Ion Non-Stick Stainless Steel Knif...,ace bergner ion non-stick stainless steel kni...,home -> kitchenware,3,None,knife,None,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179387,United Arab Emirates,TRU,6379,,6379,None,None,None,MYTS Kawasaki Styled Electric Bike Ride-on (6 ...,None,...,1.0,non_classified,TRU MYTS Kawasaki Styled Electric Bike Ride-on...,tru myts kawasaki styled electric bike ride-o...,None,0,None,None,None,0
179388,United Arab Emirates,TRU,6643,,6643,None,None,None,SMART VOICE CHANGER (BLUE),None,...,0.0,non_classified,TRU SMART VOICE CHANGER (BLUE),tru smart voice changer (blue),None,0,None,None,None,0
179389,United Arab Emirates,TRU,6846,,6846,None,None,None,"Storm 4WD RC Monster Truck (1:18), Red",None,...,0.0,non_classified,"TRU Storm 4WD RC Monster Truck (1:18), Red","tru storm 4wd rc monster truck (1:18), red",None,0,None,None,None,0
179390,United Arab Emirates,TRU,7130,,7130,None,None,None,Cam Universal Bath Stand (Silver),None,...,0.0,non_classified,TRU Cam Universal Bath Stand (Silver),tru cam universal bath stand (silver),None,0,None,None,None,0


In [123]:
df[['descr','taxonomy', 'tax_weight', 'query']].sample(10).values

array([[' stadium threadborne streaker 1/4 zip lg ', 'sport equipment',
        0, 'stadium'],
       [' bcbg eve shrt knit dress 8 ', 'clothes -> dress', 2,
        'knit dress'],
       [' stadium tsugi apex wn s 6 ',
        'sportswear -> clothes -> non categorized', 2, 'stadium s'],
       [' stadium w e 3s slim tee xl ',
        'sportswear -> clothes -> non categorized', 2, 'stadium xl'],
       [' reebok run ss tee - graphic m ',
        'sportswear -> clothes -> non categorized', 2, 'reebok m'],
       [' stadium dmx lite walk slip 8.5 ',
        'sportswear -> running shoes', 4, 'stadium 8.5'],
       [' tedbaker mi bow flip flop a38 ',
        'accessories -> wallets & handbags -> wallets', 1, 'bow'],
       [' social foot nike epic react flyknit 2 10.5 ',
        'sportswear -> running shoes', 4, 'nike 10.5'],
       [' stadium ua micro g pursuit 10 ', 'sportswear -> running shoes',
        2, 'ua 10'],
       [' quiksilver ro es ha tp j bzh0 l ', 'sportswear', 2,
        '